<a target="_blank" href="https://colab.research.google.com/github/pr4deepr/cellpose-colab/blob/main/Cellpose_cell_segmentation_2D_prediction_only.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Project 2 of classification of perturbed cells

**GOAL: Classify chemical perturbation among cells**

1. Load the dataset including 2798 data with one channel
2. Segment these images into Masks
3. Extract features from segmented images by building a CNN(UNET) to train
4. Test on example data


# Step1 Configuration



In [ ]:
pip install torchsampler


In [ ]:
## Import the usual libraries
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
from torchsampler import ImbalancedDatasetSampler
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

## print out the pytorch version used (1.31 at the time of this tutorial)
print(torch.__version__)

2.5.0+cu121


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Step2 Load the datasets

In this porject, datasets are downsampledata/(train)  + exampledata/（test, and label is in the Meta_data



In [ ]:
import re


In [ ]:
data_dir='/content/gdrive/MyDrive/Part 2/Data/downsampled_data/'
save_dir='content/gdirve/MyDrive/Project2/Data'
test_dir='/content/gdrive/MyDrive/Part 2/Data/example_data'

In [ ]:
def get_image_names(directory):
    image_names = [filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return image_names

In [ ]:
image_names = get_image_names(data_dir)
image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir) if fname.endswith('.tiff')]
image_basenames = [re.match(r'^(r\d+c\d+f\d+)', os.path.basename(fname)).group(0) for fname in image_paths]
images_df = pd.DataFrame({'image_name': image_names, 'base_name': image_basenames,'image_path':image_paths})

# Create test dataset with label
test_image_names = get_image_names(test_dir)
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.tiff')]
# test_image_basenames = [re.match(r'^(r\d+c\d+f\d+)', os.path.basename(fname)).group(0) for fname in test_image_paths]
test_df = pd.DataFrame({'test_name': test_image_names, 'label': 1,'image_path':test_image_paths})
test_df.loc[1, 'label'] = 0
test_df

,test_name,label,image_path
0,r04c08f05p01-compound-FK866.tiff,1,/content/gdrive/MyDrive/Part 2/Data/example_da...
1,r04c14f05p01-compound-DMSO.tiff,0,/content/gdrive/MyDrive/Part 2/Data/example_da...
2,r06c10f05p01-compound-quinidine.tiff,1,/content/gdrive/MyDrive/Part 2/Data/example_da...
3,r12c09f05p01-compound-FK866.tiff,1,/content/gdrive/MyDrive/Part 2/Data/example_da...
4,r13c02f05p01-compound-LY2109761.tiff,1,/content/gdrive/MyDrive/Part 2/Data/example_da...


In [ ]:
metadata_path = '/content/gdrive/MyDrive/Part 2/Data/metadata_BR00116991.csv'
metadata = pd.read_csv(metadata_path)
metadata['base_name'] = metadata['FileName_OrigRNA'].apply(lambda x: re.match(r'^(r\d+c\d+f\d+)', x).group(0))
metadata['label'] = metadata['Metadata_pert_iname'].apply(lambda x: 0 if x == 'DMSO' else 1)

filtered_metadata = metadata[metadata['base_name'].isin(image_basenames)]
filtered_metadata[['base_name','label']]



,base_name,label
0,r01c01f01,1
1,r01c01f02,1
2,r01c01f03,1
3,r01c01f04,1
4,r01c01f05,1
...,...,...
2862,r14c07f01,0
2863,r14c07f02,0
2864,r14c07f03,0
2865,r14c07f04,0


In [ ]:
label_images_df = pd.merge(images_df, metadata[['base_name', 'label']], on='base_name', how='inner')
label_images_df.drop(columns=['base_name'], inplace=True)
label_images_df.head()

,image_name,image_path,label
0,r10c01f03_median_aggregated.tiff,/content/gdrive/MyDrive/Part 2/Data/downsample...,1
1,r02c16f01_median_aggregated.tiff,/content/gdrive/MyDrive/Part 2/Data/downsample...,1
2,r05c22f06_median_aggregated.tiff,/content/gdrive/MyDrive/Part 2/Data/downsample...,1
3,r01c20f02_median_aggregated.tiff,/content/gdrive/MyDrive/Part 2/Data/downsample...,1
4,r04c11f01_median_aggregated.tiff,/content/gdrive/MyDrive/Part 2/Data/downsample...,1


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
class CustomImageDataset():
    def __init__(self, labeled_images_df, transform=None):
        self.labeled_images_df = labeled_images_df
        self.transform = transform

    def __len__(self):
        return len(self.labeled_images_df)

    def __getitem__(self, idx):
        img_path = self.labeled_images_df.iloc[idx]['image_path']
        label = self.labeled_images_df.iloc[idx]['label']  # Get binary label directly

        # Load the image
        image = Image.open(img_path).convert('L')

        # Apply transformation if specified
        if self.transform:
            image = self.transform(image)

        # Return the image and the binary label (converted to tensor)
        return image, torch.tensor(label, dtype=torch.float32).unsqueeze(0)  # Adds a channel dimension


In [ ]:
pip install scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `labeled_images_df` is your full dataset with 'image_path' and 'label' columns
train_df, val_df = train_test_split(
    label_images_df,
    test_size=0.2,
    stratify=label_images_df['label'],  # Stratify by the 'label' column
    random_state=42  # Set a random state for reproducibility
)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")


Training samples: 2293, Validation samples: 574


In [ ]:
# Initialize the dataset with no oversampling applied
train_dataset = CustomImageDataset(labeled_images_df=train_df, transform=transform)
val_dataset = CustomImageDataset(labeled_images_df=val_df, transform=transform)
test_dataset = CustomImageDataset(labeled_images_df=test_df, transform=transform)



In [ ]:
# Use ImbalancedDatasetSampler for oversampling in training

train_loader = DataLoader(train_dataset, batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader= DataLoader(test_dataset, batch_size=1, shuffle=False)

# Step3 Build an simple CNN at first




In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, 1)  # Final layer for binary output

    def forward(self, x, return_features=False):
        # First convolutional block
        x = F.relu(self.conv1(x))
        features = self.pool(x)  # Store features after first pool layer

        # Second convolutional block
        x = F.relu(self.conv2(features))
        x = self.pool(x)

        # Flatten for fully connected layers
        x = x.view(-1, 128 * 64 * 64)
        x = F.relu(self.fc1(x))

        # Final binary classification output
        output = torch.sigmoid(self.fc2(x))

        # Return both features and output if specified
        if return_features:
            return (output, features)
        else:
            return output


In [ ]:
# Initialize the model, loss function, and optimizer
model = SimpleCNN().to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Updated Training function without validation and with train accuracy
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Run the training
train_model(model, train_loader, criterion, optimizer, num_epochs=20)


Epoch [1/20], Train Loss: 17.5276, Train Accuracy: 82.25%
Epoch [2/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [3/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [4/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [5/20], Train Loss: 17.7331, Train Accuracy: 82.25%
Epoch [6/20], Train Loss: 17.8241, Train Accuracy: 82.25%
Epoch [7/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [8/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [9/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [10/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [11/20], Train Loss: 17.6877, Train Accuracy: 82.25%
Epoch [12/20], Train Loss: 17.7104, Train Accuracy: 82.25%
Epoch [13/20], Train Loss: 17.8241, Train Accuracy: 82.25%
Epoch [14/20], Train Loss: 17.7331, Train Accuracy: 82.25%
Epoch [15/20], Train Loss: 17.7104, Train Accuracy: 82.25%
Epoch [16/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [17/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch 

In [ ]:
# Updated Training function with feature extraction
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass with feature extraction
            outputs, features = model(images, return_features=True)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

    return features  # Return features of the last batch

In [ ]:
# Run the training
train_features = train_model(model, train_loader, criterion, optimizer, num_epochs=20)


Epoch [1/20], Train Loss: 17.6877, Train Accuracy: 82.25%
Epoch [2/20], Train Loss: 17.7331, Train Accuracy: 82.25%
Epoch [3/20], Train Loss: 17.7331, Train Accuracy: 82.25%
Epoch [4/20], Train Loss: 17.8013, Train Accuracy: 82.25%
Epoch [5/20], Train Loss: 17.8241, Train Accuracy: 82.25%
Epoch [6/20], Train Loss: 17.7104, Train Accuracy: 82.25%
Epoch [7/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [8/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [9/20], Train Loss: 17.7104, Train Accuracy: 82.25%
Epoch [10/20], Train Loss: 17.7104, Train Accuracy: 82.25%
Epoch [11/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [12/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch [13/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [14/20], Train Loss: 17.7559, Train Accuracy: 82.25%
Epoch [15/20], Train Loss: 17.7331, Train Accuracy: 82.25%
Epoch [16/20], Train Loss: 17.8013, Train Accuracy: 82.25%
Epoch [17/20], Train Loss: 17.7786, Train Accuracy: 82.25%
Epoch 

In [ ]:
len(train_features)

21

In [ ]:
train_features

tensor([[[[8.2440e-01, 7.7216e-01, 7.6527e-01,  ..., 7.3772e-01,
           7.4964e-01, 7.4436e-01],
          [7.7224e-01, 7.7126e-01, 7.6987e-01,  ..., 7.5116e-01,
           7.5290e-01, 7.4705e-01],
          [7.6989e-01, 7.6966e-01, 7.6839e-01,  ..., 7.5623e-01,
           7.5185e-01, 7.4936e-01],
          ...,
          [7.7204e-01, 7.7204e-01, 7.7204e-01,  ..., 6.4628e-01,
           6.8819e-01, 7.2345e-01],
          [7.7204e-01, 7.7204e-01, 7.7204e-01,  ..., 6.5703e-01,
           6.9978e-01, 6.9481e-01],
          [7.7204e-01, 7.7204e-01, 7.7204e-01,  ..., 6.3940e-01,
           6.4733e-01, 6.5991e-01]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.0201e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.0703e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.0831e-01],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000

In [ ]:
# Set up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate and load your trained model
model = SimpleCNN().to(device)  # Move model to the GPU
# Load trained weights if available
# model.load_state_dict(torch.load('your_model.pth'))

# Switch to evaluation mode
model.eval()

# Hook function to capture features from the last conv layer
def get_features(module, input, output):
    model.features = output

# Register hook on the last convolutional layer
model.conv2.register_forward_hook(get_features)

# Create a sample image and move it to the GPU
sample_image = torch.randn(1, 1, 128, 128).to(device)  # Example input size; use your actual input here
with torch.no_grad():
    _ = model(sample_image)  # Forward pass

# Extract the features and move to CPU for visualization
features = model.features.squeeze(0).cpu()  # Remove batch dimension and move to CPU

### Visualize Feature Maps

# Plot each feature map as a grayscale image
num_features = features.size(0)  # Number of feature maps
plt.figure(figsize=(15, 15))
for i in range(num_features):
    plt.subplot(8, 8, i + 1)  # Adjust grid size as needed, e.g., 8x8 for 64 features
    plt.imshow(features[i].numpy(), cmap='gray')
    plt.axis('off')
plt.show()

RuntimeError: shape '[-1, 524288]' is invalid for input of size 131072

In [ ]:
def test_single_images(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = len(test_loader.dataset)  # Total number of images in the test dataset
    predictions = []

    with torch.no_grad():  # Disable gradient computation for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass to get predictions
            outputs = model(images)
            predicted = (outputs > 0.5).float()  # Threshold at 0.5 for binary prediction

            # Collect predictions for display
            predictions.extend(predicted.cpu().numpy())

            # Calculate the number of correct predictions
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f"Test Accuracy on {total} images: {accuracy:.2f}%")
    return predictions



In [ ]:
# Run the test
predictions = test_single_images(model, test_loader, device)

# Display predictions (optional)
for i, pred in enumerate(predictions):
    print(f"Image {i+1}: Predicted Label = {int(pred)}, Ground Truth = {test_loader.dataset[i][1].item()}")


Test Accuracy on 5 images: 80.00%
Image 1: Predicted Label = 1, Ground Truth = 1.0
Image 2: Predicted Label = 1, Ground Truth = 0.0
Image 3: Predicted Label = 1, Ground Truth = 1.0
Image 4: Predicted Label = 1, Ground Truth = 1.0
Image 5: Predicted Label = 1, Ground Truth = 1.0


<ipython-input-22-fdb6f29fd06f>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Image {i+1}: Predicted Label = {int(pred)}, Ground Truth = {test_loader.dataset[i][1].item()}")


In [ ]:
def test_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_features = []
    all_outputs = []

    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs, features = model(images, return_features=True)
            all_features.append(features.cpu())  # Collect features
            all_outputs.append(outputs.cpu())    # Collect outputs

    return torch.cat(all_features), torch.cat(all_outputs)  # Return all features and outputs

In [ ]:
# Run the testing and extract features
test_model(model, test_loader)

# Step4 Segmentation before CNN

In [ ]:
import cv2

def segment_cells(image):
    # Convert PIL image to a numpy array
    image_np = np.array(image)

    # Apply a binary threshold to create a mask
    _, mask = cv2.threshold(image_np, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find contours of cells
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract each cell as a sub-image
    cell_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cell_image = image_np[y:y+h, x:x+w]  # Crop each cell region
        cell_images.append(Image.fromarray(cell_image))

    return cell_images

In [ ]:
class CellSegmentationDataset():
    def __init__(self, labeled_images_df, transform=None):
        self.labeled_images_df = labeled_images_df
        self.transform = transform

    def __len__(self):
        return len(self.labeled_images_df)

    def __getitem__(self, idx):
        img_path = self.labeled_images_df.iloc[idx]['image_path']
        label = self.labeled_images_df.iloc[idx]['label']

        # Load the original image and segment cells
        image = Image.open(img_path).convert('L')
        cell_images = segment_cells(image)  # List of cell images

        # Apply transformations and create a batch of cells with labels
        transformed_cells = []
        for cell_image in cell_images:
            if self.transform:
                cell_image = self.transform(cell_image)
            transformed_cells.append((cell_image, torch.tensor(label, dtype=torch.float32)))

        return transformed_cells  # List of (cell_image, label) pairs


In [ ]:
def collate_fn(batch):
    # Flatten the list of cell images and labels from all images in the batch
    cell_images, labels = zip(*[cell for sublist in batch for cell in sublist])
    return torch.stack(cell_images), torch.tensor(labels)

# Initialize the datasets and dataloaders
train_seg_dataset = CellSegmentationDataset(labeled_images_df=train_df, transform=transform)
val_seg_dataset = CellSegmentationDataset(labeled_images_df=val_df, transform=transform)
test_seg_dataset = CellSegmentationDataset(labeled_images_df=test_df, transform=transform)

# Create dataloaders for segmentation
train_seg_loader = DataLoader(train_seg_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_seg_loader = DataLoader(val_seg_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_seg_loader= DataLoader(test_seg_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [ ]:
train_model(model, train_seg_loader, criterion, optimizer, num_epochs=20)


OutOfMemoryError: CUDA out of memory. Tried to allocate 227.47 GiB. GPU 0 has a total capacity of 14.75 GiB of which 6.53 GiB is free. Process 3729 has 8.21 GiB memory in use. Of the allocated memory 7.57 GiB is allocated by PyTorch, and 516.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

I also write a train function with validation part, but the image dataset is so big, it is hard to run the train-function with validation set to get the results. Therefore, I pass this part.

In [ ]:
# Training function  with validation
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")

        # Validation after each epoch
        validate_model(model, test_loader)

# Validation function
def validate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    running_val_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            # Apply threshold to get binary predictions
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = running_val_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Run the training
#train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)


In [ ]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            # `batch` is a list of segmented cells and their labels for a single test image
            for cell_image, label in batch[0]:  # batch[0] to unpack list of cells for this batch
                cell_image = cell_image.to(device).unsqueeze(0)  # Add batch dimension
                label = label.to(device).unsqueeze(0)  # Ensure label is in [1] shape

                # Forward pass and prediction
                output = model(cell_image)
                predicted = (output > 0.5).float()  # Binary thresholding

                # Update metrics
                correct += (predicted == label).sum().item()
                total += 1

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")




In [ ]:
model = SimpleCNN().to(device)
model.load_state_dict(torch.load("model.pth"))  # Load pre-trained model weights if saved
test_model(model, test_loader, device)